# Conversational Chatbot

**Following below is the entire code for a basic chatbot model. Some code has been removed after execution. Your job is to complete the code so that it gives the corresponsing suitable output.**

Install any dependencies and libraries as required. Most of them work with a simple 'pip install' in the terminal window.

Remember to go through the intense.json file carefully to understand its components well.

If you are using Colab, then no dependencies need to be downloaded.

In [3]:
# Import the required modules
# Nothing to be added or changed here.

import random
import json
import pickle
import numpy as np
import nltk
import tensorflow as tf

# NLTK

For this chatbot, we employ the Natural Language ToolKit (NLTK). NLTK is a suite of libraries and programs for symbolic and statistical natural language processing for English. It is very commonly used to process English words and work with human language data. It helps us to break down and process the input text and resolve it suitably into the desired output.

In [4]:
# ONLY IF YOU ARE USING COLAB. Upload intense.json to your drive and mount the drive to access it
# Nothing to be added or changed here.

# from google.colab import drive
# drive.mount('/content/my-drive')

## Now begins the code to train our language model

In [5]:
# Import the required modules
# Nothing to be added or changed here.

from keras.models import Sequential
from nltk.stem import WordNetLemmatizer
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import gradient_descent_v2

In [6]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Krishna\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [7]:
# Create a lemmatizer. This later helps in processing the input text. 
# Nothing to be added or changed here.

lemmatizer = WordNetLemmatizer()

In [8]:
# Read the json.intense file
# Nothing to be added or changed here.

intents = json.loads(open("./intense.json").read())

In [9]:
# Creating empty lists to store data
# Nothing to be added or changed here.

words = []
classes = []
documents = []
ignore_letters = ["?", "!", ".", ","]
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # separating words from patterns
        word_list = nltk.word_tokenize(pattern)
        words.extend(word_list)  # and adding them to words list
          
        # associating patterns with respective tags
        documents.append(((word_list), intent['tag']))
  
        # appending the tags to the class list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [10]:
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Krishna\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Krishna\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [11]:
# Store the root words or lemma using the lemmatizer
# Nothing to be added or changed here.

words = [lemmatizer.lemmatize(word)
         for word in words if word not in ignore_letters]
words = sorted(set(words))

In [12]:
# Save the words and classes list to binary files
# Nothing to be added or changed here.

pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))

Now, we convert the text into numerical form to make it readible to the neural network that will be created

In [13]:
# No changes here

training = []
output_empty = [0]*len(classes)
for document in documents:
    bag = [] # Recall Bag of Words!
    word_patterns = document[0]
    word_patterns = [lemmatizer.lemmatize(
        word.lower()) for word in word_patterns]
    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)
          
    # Make a copy of the output_empty
    output_row = list(output_empty)
    output_row[classes.index(document[1])] = 1
    training.append([bag, output_row])
random.shuffle(training)
training = np.array(training)

C:\Users\Krishna\AppData\Local\Temp\ipykernel_16992\2258295257.py:18: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


# Now is where the real training code starts!

In [14]:
# Split the data into train_x and train_y
# your code here
# print(training.shape)
train_x = [x for x, y in training]
train_y = [y for x, y in training]
print(len(train_x[15]))

33


In [15]:
# Create a Neural network using a Sequential layer, followed by a few alternating dense and dropout layers. 
# Use activation function 'relu' and a dropout of 0.5. For the last dense layer, use 'softmax' activation function
# your code here
model = Sequential()
model.add(Dense(64, input_shape=(17,33), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))

In [16]:
# Compile the model
# No changes here 

sgd = gradient_descent_v2.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',
              optimizer=sgd, metrics=['accuracy'])
hist = model.fit(np.array(train_x), np.array(train_y),
                 epochs=200, batch_size=5, verbose=1)

Epoch 1/200


c:\Users\Krishna\anaconda3\envs\tf-gpu\lib\site-packages\keras\optimizer_v2\optimizer_v2.py:355: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


4/4 [==============================] - 0s 1ms/step - loss: 1.2116 - accuracy: 0.2353
Epoch 2/200
4/4 [==============================] - 0s 1ms/step - loss: 1.1337 - accuracy: 0.2941
Epoch 3/200
4/4 [==============================] - 0s 1ms/step - loss: 1.1271 - accuracy: 0.2353
Epoch 4/200
4/4 [==============================] - 0s 1ms/step - loss: 1.1321 - accuracy: 0.3529
Epoch 5/200
4/4 [==============================] - 0s 1ms/step - loss: 1.0312 - accuracy: 0.5882
Epoch 6/200
4/4 [==============================] - 0s 1ms/step - loss: 1.0804 - accuracy: 0.3529
Epoch 7/200
4/4 [==============================] - 0s 1ms/step - loss: 1.0681 - accuracy: 0.4706
Epoch 8/200
4/4 [==============================] - 0s 2ms/step - loss: 0.9599 - accuracy: 0.7059
Epoch 9/200
4/4 [==============================] - 0s 1ms/step - loss: 0.9931 - accuracy: 0.5882
Epoch 10/200
4/4 [==============================] - 0s 1ms/step - loss: 1.0846 - accuracy: 0.4706
Epoch 11/200
4/4 [=======================

In [17]:
# Save the model for use ahead
# No changes here

model.save("chatbotmodel.h5", hist)
  
# Print a message for the successful training of the Chatbot model
print("Model saved!")

Model saved!


# And finally, we execute the Chatbot Model

In [18]:
# No change

from keras.models import load_model

In [19]:
# Load the files we made previously - 
# load 'intense.json' into variable 'intents'
# load 'words.pk1' into variable 'words'
# load 'classes.pk1' into variable 'classes'
# load 'chatbotmodel.h5' into variable 'model'
with open('intense.json', 'r') as file:
    intents = json.load(file)

with open('words.pkl', 'rb') as file:
    words = pickle.load(file)

with open('classes.pkl', 'rb') as file:
    data = pickle.load(file)

model = load_model('chatbotmodel.h5')



In [20]:
# Create a function 'clean_up_sentences(sentence)' to separate words from the input sentences (this is called tokenisation). 
# Use nltk.word_tokenize(sentence) and return the words of the sentence after lemmatization

# your code here

def clean_up_sentences(sentence):
    tokens = nltk.word_tokenize(sentence)
    lemmatizer = WordNetLemmatizer()
    lemmas = [lemmatizer.lemmatize(token) for token in tokens]
    return lemmas

In [21]:
# Create a function 'bagw(sentence)' to  append 1 to a list variable ‘bag’ if the word is contained inside our input and is also present in the list of words created earlier.
# Return a numpy array of the list variable bag that now contains 1’s and 0’s.

def bagw(sentence):
    
    # Separate out words from the input sentence
    sentence_words = clean_up_sentences(sentence)
    bag = [0]*len(words)
    for w in sentence_words:
        for i, word in enumerate(words):
  
            # check whether the word
            # is present in the input as well
            
            #your code here
            if word == w:
                bag[i] = 1
                break
            
            
    # return a numpy array
    return np.array(bag)

In [22]:
# Create a function 'predict_class(sentence)' to predict the class of the sentence input by the user.

def predict_class(sentence):
  # Initialize a variable 'bow' that will contain a NumPy array of 0’s and 1’s, using the bag(w) function defined above
  # Your line of code here 
    bow = bagw(sentence)
    res = model.predict(np.array([bow]))[0]

    # In variable 'results', append from ‘res’ if the value is greater than the ERROR_THRESHOLD, then sort it using the sort function. 
    # No change
    ERROR_THRESHOLD = 0.25
    results = [[i, r] for i, r in enumerate(res) 
               if r > ERROR_THRESHOLD]
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({'intent': classes[r[0]],
                            'probability': str(r[1])})
    return return_list

In [23]:
# Create 'get_response(intents_list, intents_json)' to generate a random response from whichever class the sentence/words input by the user belongs to.

def get_response(intents_list, intents_json):
    tag = intents_list[0]['intent']
    list_of_intents = intents_json['intents']
    result = ""
    for i in list_of_intents:
    # If the tag matches the tag in list_of_intents, store a random response from the 'intense.json' file in a variable called 'result'
    # your code here
        if tag == i:
            result = intents_json[int(Math.random()*len(intents_json))]['responses']        
    return result
  
print("Chatbot is up!")

Chatbot is up!


# And our Chatbot is ready! Time to get it into action

In [26]:
# Create an infinite loop to keep the model running.
# No edits here

while True:
    message = input("")
    ints = predict_class(message)
    print(ints)
    res = get_response(ints, intents)
    print(res)
    # Write a couple of lines of code to stop the infinite loop if the intent of the message is 'goodbye'
    if ints[0]['intent'] == 'goodbye':
        break

[{'intent': 'greeting', 'probability': '0.82757616'}]

[{'intent': 'goodbye', 'probability': '0.9962465'}]

